In [139]:
import json
import networkx as nx
import dynetx as dn
import re
import numpy as np
from scipy.stats import poisson

In [163]:
def understand() :
    G = nx.Graph()
    G.add_edge(2, 3)
    G.add_edge(1, 2)

    DG = dn.DynGraph()
    DG.add_interactions_from(G.edges(), t=0)
    DG.add_interactions_from([(1,2)],t=1)
    DG.add_interactions_from([(1,2)],t=2)
    DG.add_interactions_from([(2,3)],t=2)
    DG.add_interactions_from([(1,3)],t=0)
    DG.add_interactions_from([(1,3)],t=1)
    DG.add_interactions_from([(1,3)],t=2)
    DG.add_interactions_from([(1,3)],t=3)
    DG.add_interactions_from([(1,4)],t=4)
    # print(DG.interactions())
    # print(DG.interactions(t=0))
    G = nx.Graph([(edge[0], edge[1]) for edge in DG.interactions(t=0)])
    print(nx.degree(G))


    # print(DG.time_slice(t_from=0, t_to=4).nodes())
    # print(DG.time_slice(t_from=0, t_to=1).degree())
    

    # for e in DG.stream_interactions():
    #     print(e)
    
    # print(DG.time_slice(t_from=0, t_to=2).edges)
    # print(DG.time_slice(3).edges)
    # g = nx.MultiGraph(DG.time_slice(t_from=0, t_to=2).edges)
    # print(g.edges())
    # nx.draw_circular(g, with_labels=True)
    
understand()

[(2, 2), (3, 2), (1, 2)]


In [2]:
def readPreferenceGraph(filePath) :
    with open("./preferenceGraph.json") as file :
        data = json.load(file)

    G = nx.DiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], comment=edge["comment"])
    
    return G

In [242]:
def interactionGraph(preferenceGraph, personalityThreshold) :
   def calculateProbability(weight) :
      if re.match(r".*prefers.*collaborate.*|.*prefers.*socialize.*", weight) :
            return 1
      elif re.match(r".*perceives.*prefers.*collaborate.*|.*perceives.*prefers.*socialize.*", weight) :
         return 0.7
      elif re.match(r".*perceives.*avoids.*collaborate.*|.*perceives.*avoids.*socialize.*", weight) :
         return 0.3
      else :
         return 0

   def analyzePreferenceGraph(timestamp) :
      
      #dictionary {edge : probability of existing in the Preference Graph}
      edgesProbabilities = {}

      #examine each eage in Preference Graph
      #calculate probability
      for u, v in preferenceGraph.edges() :
         if preferenceGraph.has_edge(v, u) :
            #(u,v) and (v,u) are checked together once
            if (v,u) in edgesProbabilities :
                #(v,u) is already checked, so neglect (u,v)
               continue 
            else :
               #(u,v), (v,u) pair has not been checked
               edgesProbabilities[(u,v)] = 0.5*(calculateProbability(preferenceGraph[u][v]["comment"]) + calculateProbability(preferenceGraph[v][u]["comment"]))
         else :
            edgesProbabilities[(u,v)] = calculateProbability(preferenceGraph[u][v]["comment"])

      #create edge in Interaction Graph with respect to the existing edge probability
      for edge, prob in edgesProbabilities.items() :
         if np.random.choice([0,1], p=[1-prob, prob]) == 1 :
            interactionGraph.add_interaction(edge[0], edge[1], timestamp)
      
   def analyzeStudentsPreferences(timestamp) :

      #get a snapshot at the timestamp where students' preferences were added
      snapshot = nx.Graph()
      snapshot.add_nodes_from(range(preferenceGraph.number_of_nodes()))
      snapshot.add_edges_from([(edge[0], edge[1]) for edge in interactionGraph.interactions(t=timestamp)])
      
      #calculate degree of nodes
      #detect the maximum of them
      #find the ratio of a node over the maximum
      snapshotDegrees = snapshot.degree()
      maxDegree = max(snapshotDegrees, key=lambda item:item[1])[1]
      nodesRating = [(element[0], element[1]/maxDegree) for element in snapshotDegrees]
      
      #split students in low and high profile based on a threshold
      lowProfileStudents = set()
      highProfileStudents = set()
      for node, degree in nodesRating :
         if degree < personalityThreshold :
            lowProfileStudents.add(node)
         else :
            highProfileStudents.add(node)
      
      #distribution of each students' category
      lowProfileDistribution = poisson.pmf(range(5), mu=0.8)
      highProfileDistribution = poisson.pmf(range(5), mu=1.5)

      def checkProbSum(arr) :
         diff = 1-sum(arr)
         if diff > 0 :
            arr[-1] += abs(diff)
         elif diff < 0 :
            arr[-1] -= abs(diff)
         else :
            None
         return arr

      return lowProfileStudents, highProfileStudents, checkProbSum(lowProfileDistribution), checkProbSum(highProfileDistribution)

   def createRandomEdgesAtTimestamp(timestamp) :
      nodes = preferenceGraph.nodes()
      for node in nodes :
         distribution = lowProfileDistribution if node in lowProfileStudents else highProfileDistribution
         numberOfInteractions = np.random.choice(range(len(distribution)), p=distribution)
         if numberOfInteractions == 0 :
            continue

         count = 0
         while True :
            newInteraction = np.random.choice(nodes)
            if not(newInteraction == node) :
               count += 1
               interactionGraph.add_interaction(node, newInteraction, t=timestamp)

            if count == numberOfInteractions :
               break
         
                  
   #create an empty Interaction Graph
   interactionGraph = dn.DynGraph()
   
   #interactions at timetsamp t = 0
   #interactions based on a scale-free network model
   G = nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), m=3)
   interactionGraph.add_interactions_from(G.edges(), t=0)
   

   #interactions at timestamp t = 1
   #interactions based on Preference Graph (students' preferences for their classmates)
   analyzePreferenceGraph(timestamp=1)


   #analyze students' preferences
   #Based on the personalityThreshold, split students in 2-categories: low profile and high profile
   lowProfileStudents, highProfileStudents, lowProfileDistribution, highProfileDistribution = analyzeStudentsPreferences(timestamp=1)
   
   
   #interactions at timestamp t = 2
   createRandomEdgesAtTimestamp(2)
   
   
    
   return interactionGraph





path = "./preferenceGraph.json"
interactionGraph(preferenceGraph=readPreferenceGraph(path), personalityThreshold=0.4)


In [ ]:
#take into account the classmates that the nodes want to avoid